In [0]:
import os
#import pandas as pd
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split,  cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.externals import joblib

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier # keras wrapper for sklearn

import json
import zipfile

In [29]:
######################################## ERASE THE KEY EVERYTIME
!pip install kaggle
api_token = {"username":"jpinzon","key":"KEY HERE"}
import json
import zipfile
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [30]:
if not os.path.exists("/TK_Model"):
    os.makedirs("/TK_Model")
os.chdir('/TK_Model')
os.listdir('.')

[]

In [31]:
!kaggle competitions download  talkingdata-adtracking-fraud-detection

100% 1.20G/1.21G [00:20<00:00, 39.8MB/s]
100% 1.21G/1.21G [00:20<00:00, 62.4MB/s]
  0% 0.00/1.08M [00:00<?, ?B/s]
100% 1.08M/1.08M [00:00<00:00, 143MB/s]
 99% 164M/165M [00:04<00:00, 25.5MB/s]
100% 165M/165M [00:04<00:00, 35.5MB/s]
 90% 36.0M/39.8M [00:00<00:00, 17.7MB/s]
100% 39.8M/39.8M [00:00<00:00, 43.2MB/s]
 96% 481M/504M [00:08<00:00, 48.9MB/s]
100% 504M/504M [00:08<00:00, 64.4MB/s]


In [32]:
zipfile.ZipFile('train_sample.csv.zip').extractall('train_sample.csv')
os.listdir('train_sample.csv/mnt/ssd/kaggle-talkingdata2/competition_files/')

['train_sample.csv']

In [0]:
data_s = pd.read_csv('train_sample.csv/mnt/ssd/kaggle-talkingdata2/competition_files/train_sample.csv', engine='python')
#.fillna('')

In [34]:
data_s.sort_values(by='attributed_time', ascending = False).head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
28970,346097,19,16,0,213,2017-11-09 15:27:07,2017-11-09 15:28:15,1
5111,303875,71,1,19,3,2017-11-09 15:13:48,2017-11-09 15:17:13,1
93842,346361,115,1,22,203,2017-11-09 15:14:10,2017-11-09 15:15:34,1
39247,305220,19,581,24,213,2017-11-09 15:03:23,2017-11-09 15:05:43,1
27035,75443,9,1,19,466,2017-11-09 14:12:55,2017-11-09 14:53:22,1


In [0]:
#import dask.dataframe as dd
#data_s = dd.read_csv('train_sample.csv')
#results=data_s.groupby('user').apply(calc_HR_ind_dsk).compute()

In [36]:
data_s

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0
5,93663,3,1,17,115,2017-11-09 01:22:13,NaN,0
6,17059,1,1,17,135,2017-11-09 01:17:58,NaN,0
7,121505,9,1,25,442,2017-11-07 10:01:53,NaN,0
8,192967,2,2,22,364,2017-11-08 09:35:17,NaN,0
9,143636,3,1,19,135,2017-11-08 12:35:26,NaN,0


In [37]:
data_s.sort_values(by='attributed_time', ascending = False).head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
28970,346097,19,16,0,213,2017-11-09 15:27:07,2017-11-09 15:28:15,1
5111,303875,71,1,19,3,2017-11-09 15:13:48,2017-11-09 15:17:13,1
93842,346361,115,1,22,203,2017-11-09 15:14:10,2017-11-09 15:15:34,1
39247,305220,19,581,24,213,2017-11-09 15:03:23,2017-11-09 15:05:43,1
27035,75443,9,1,19,466,2017-11-09 14:12:55,2017-11-09 14:53:22,1


In [38]:
# Checking for NANs
for col in data_s.columns:
    print('{} contains {:,} NANs'.format(col,len(data_s[data_s[col].isnull()])))

ip contains 0 NANs
app contains 0 NANs
device contains 0 NANs
os contains 0 NANs
channel contains 0 NANs
click_time contains 0 NANs
attributed_time contains 99,773 NANs
is_attributed contains 0 NANs


Only attributed time has NANs, but I am not using this column

In [0]:
# Removing attributed_time
data_s = data_s.drop('attributed_time', axis = 1)

In [40]:
data_s.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [0]:
# Convert click_time to datetime format
data_s.click_time = pd.to_datetime(data_s.click_time, errors = 'ignore')

In [0]:
# Determine and coding time of the day the site was accessed
# 1 = Night, 2 = Morning, 3 = Afternoon, 4 = Evening
data_s = data_s.assign(session = pd.cut(data_s.click_time.dt.hour,[-1,6,12,18,24],labels=[1, 2, 3,4 ]))
#data_s.session = data_s.session.astype(np.int64)
#data_s = data_s.drop('click_time', 1)

In [43]:
data_s.head()

,ip,app,device,os,channel,click_time,is_attributed,session
0,87540,12,1,13,497,2017-11-07 09:30:38,0,2
1,105560,25,1,17,259,2017-11-07 13:40:27,0,3
2,101424,12,1,19,212,2017-11-07 18:05:24,0,3
3,94584,13,1,13,477,2017-11-07 04:58:08,0,1
4,68413,12,1,1,178,2017-11-09 09:00:09,0,2


In [0]:
data_s = data_s[['ip', 'app', 'device', 'os', 'channel', 'session','is_attributed']]

In [0]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
# Encoding the variable
fit = data_s.apply(lambda x: d[x.name].fit_transform(x))

# Inverse the encoded
#fit.apply(lambda x: d[x.name].inverse_transform(x))
# Using the dictionary to label future data
#data_s.apply(lambda x: d[x.name].transform(x))

In [46]:
data_s.head()

,ip,app,device,os,channel,session,is_attributed
0,87540,12,1,13,497,2,0
1,105560,25,1,17,259,3,0
2,101424,12,1,19,212,3,0
3,94584,13,1,13,477,1,0
4,68413,12,1,1,178,2,0


In [0]:
data_s = data_s.reset_index(drop = False)
data_s = data_s.rename(columns={'index':'click_id'})

In [48]:
data_s.is_attributed.unique()

array([0, 1])

In [49]:
data_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
click_id         100000 non-null int64
ip               100000 non-null int64
app              100000 non-null int64
device           100000 non-null int64
os               100000 non-null int64
channel          100000 non-null int64
session          100000 non-null category
is_attributed    100000 non-null int64
dtypes: category(1), int64(7)
memory usage: 5.4 MB


In [50]:
data_s.sort_values(by='ip').head()

,click_id,ip,app,device,os,channel,session,is_attributed
4249,4249,9,9,1,13,244,3,0
51193,51193,10,18,1,13,107,2,0
18992,18992,10,11,1,22,319,1,0
25399,25399,10,12,1,19,140,2,0
10686,10686,19,14,1,16,379,2,0


# Aritificial Neural Network 

### Split data into target and features

In [51]:
target = data_s.is_attributed.values
features = data_s.drop('is_attributed', 1).values
print ('Data original %d, target: %d, features: %d' % (data_s.shape[0], target.shape[0], features.shape[0]))

Data original 100000, target: 100000, features: 100000


### Split features into training, validation, and testing sets

In [0]:
X_train, X_test, y_train, y_test  = train_test_split(features, target, test_size= 0.25, random_state=1)
X_train, X_val, y_train, y_val    = train_test_split(X_train, y_train, test_size= 0.20, random_state=1)

X_train_names = X_train[:,0]
X_train = X_train[:,1:7]

X_test_names = X_test[:,0]
X_test = X_test[:,1:7]
                  
X_val_names = X_val[:,0]
X_val = X_val[:,1:7]

### Standarizing the features

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val   = sc.transform(X_val)
X_test  = sc.transform(X_test)

In [55]:
X_train

array([[-1.04568937e+00, -6.12638678e-01, -7.93763116e-02,
        -8.59872320e-02,  1.33581785e+00, -2.31288181e-02],
       [ 5.00223830e-01, -6.12638678e-01, -7.93763116e-02,
         1.69632802e-01,  8.60994279e-02, -1.05260143e+00],
       [ 1.45368304e+00, -6.12638678e-01, -7.93763116e-02,
         2.24408523e-01,  1.08124558e+00, -1.05260143e+00],
       ...,
       [-1.10247574e+00, -7.48701236e-01, -7.54766277e-02,
        -1.77280101e-01, -1.04019002e+00, -2.31288181e-02],
       [-9.47442656e-01, -3.57164216e-04, -7.93763116e-02,
        -1.77280101e-01, -7.59011084e-02, -2.31288181e-02],
       [ 3.85607429e-01,  2.03736674e-01, -7.93763116e-02,
        -1.77280101e-01,  1.24324612e+00, -1.05260143e+00]])

### Building the initial model 

In [0]:
def classifier_model(activator, optimizer, initializer):
    classifier = Sequential()
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator, input_dim = 6))
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator))
    classifier.add(Dense(units = 1,  kernel_initializer = initializer, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

### Fitting the model with the training set 

In [57]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
classifier = classifier_model(activator, optimizer, initializer)
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
60000/60000 [==============================] - 30s 494us/step - loss: 0.0305 - acc: 0.9977
Epoch 2/10
60000/60000 [==============================] - 28s 467us/step - loss: 0.0130 - acc: 0.9978
Epoch 3/10
60000/60000 [==============================] - 28s 463us/step - loss: 0.0127 - acc: 0.9978
Epoch 4/10
60000/60000 [==============================] - 28s 464us/step - loss: 0.0126 - acc: 0.9978
Epoch 5/10
60000/60000 [==============================] - 28s 467us/step - loss: 0.0126 - acc: 0.9978
Epoch 6/10
60000/60000 [==============================] - 28s 465us/step - loss: 0.0124 - acc: 0.9978
Epoch 7/10
60000/60000 [==============================] - 28s 463us/step - loss: 0.0124 - acc: 0.9978
Epoch 8/10
60000/60000 [==============================] - 28s 465us/step - loss: 0.0124 - acc: 0.9978
Epoch 9/10
60000/60000 [==============================] - 28s 462us/step - loss: 0.0124 - acc: 0.9978
Epoch 10/10
60000/60000 [==============================] - 28s 464us/step - loss: 

### Crossvalidation on training set

In [0]:
# Using relu - as raw model above - as activator
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
cv = 10 
classifier_cv = KerasClassifier(build_fn = classifier_model, 
                                activator = activator, 
                                optimizer = optimizer,
                                initializer = initializer,
                                batch_size = 10, 
                                epochs = 10, 
                                verbose = 0)
accuracies = cross_val_score(estimator = classifier_cv, X = X_train, y = y_train, cv = cv)

mean = accuracies.mean()
variance = accuracies.std()

In [0]:
# Mean cv accuracy 
print('Mean cv accuracy (relu) = {:.4f}% +/- {:.4f}'.format(mean *100, variance *100) )

### Validation 

In [0]:
loss, accuracy = classifier.evaluate(X_val, y_val, batch_size = 10, verbose = 0)
print("Validation set accuracy = {:.4f}%, Loss = {:.4f}".format(accuracy* 100, loss))

Validation set accuracy = 99.7533%, Loss = 0.0133


In [0]:
loss, accuracy = classifier.evaluate(X_train, y_train , batch_size = 128, verbose = 0)
print(round(accuracy*100,2))

99.78


### Optimization

Using GridSearch

In [0]:
classifier_op = KerasClassifier(build_fn = classifier_model)

parameters = {'batch_size'  : [10, 30],
              'epochs'      : [10, 20],
              'optimizer'   : ['adam', 'rmsprop'],
              'activator'   : ['relu', 'sigmoid'],
              'initializer' : ['uniform']}

grid_search = GridSearchCV(estimator = classifier_op,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/10
54000/54000 [==============================] - 7s 125us/step - loss: 0.0352 - acc: 0.9976
Epoch 2/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0128 - acc: 0.9978
Epoch 3/10
54000/54000 [==============================] - 6s 111us/step - loss: 0.0125 - acc: 0.9978
Epoch 4/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0124 - acc: 0.9978
Epoch 5/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0123 - acc: 0.9978
Epoch 6/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0123 - acc: 0.9978
Epoch 7/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0122 - acc: 0.9978
Epoch 8/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0122 - acc: 0.9978
Epoch 9/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0121 - acc: 0.9978
Epoch 10/10
54000/54000 [==============================] - 6s 110us/step - loss: 0.0121 - a

In [0]:
best_accuracy

In [0]:
best_parameters

# OPTIMIZED MODEL 

In [0]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
epoch = 10 
classifier_opt = classifier_model(activator, optimizer, initializer)
classifier_opt.fit(X_train, y_train, batch_size = 10, epochs = epoch, verbose = 0)

In [0]:
# Predicting on the test set 
y_pred = classifier_opt.predict(X_test)
y_pred 

In [0]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred.round(), average='weighted')

In [0]:
y_pred.round()

In [0]:
y_classes = y_pred.argmax(axis=-1)
y_a = pd.DataFrame([y_classes])
y_b = pd.DataFrame([y_test])
y_c = pd.DataFrame([X_test_names])
y_a = pd.concat((y_a,y_b, y_c)).T
y_a.columns = ['predicted', 'actual','click_id']
y_a.head()

In [0]:
# Number of mismatches within the test set
y_a['dif'] = np.where(y_a.predicted == y_a.actual, 0, 1)
y_a[y_a['dif'] == 1].actual.value_counts().sort_index()

In [0]:
classifier_opt.save('classifier.hdf5')
joblib.dump(sc, 'scaler.pkl') 

------------------
PREDICTION ON THE TESTING DATASET

In [0]:
from keras.models import load_model
classifier = load_model('classifier.hdf5')
from sklearn.preprocessing import StandardScaler
sc = joblib.load('scaler.pkl')

In [0]:
test = pd.read_csv('test.csv')

In [0]:
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [0]:
# Convert click_time to datetime format
test.click_time = pd.to_datetime(test.click_time, errors = 'ignore')

In [0]:
# Checking for NANs
for col in test.columns:
    print('{} contains {:,} NANs'.format(col,len(test[test[col].isnull()])))

click_id contains 0 NANs
ip contains 0 NANs
app contains 0 NANs
device contains 0 NANs
os contains 0 NANs
channel contains 0 NANs
click_time contains 0 NANs


In [0]:
# Determine and coding time of the day the site was accessed
# 1 = Night, 2 = Morning, 3 = Afternoon, 4 = Evening
test = test.assign(session=pd.cut(test.click_time.dt.hour,[-1,6,12,18,24],labels=[1, 2, 3,4 ]))
test = test.drop('click_time', 1)

In [0]:
test.columns

Index(['click_id', 'ip', 'app', 'device', 'os', 'channel', 'session'], dtype='object')

In [0]:
test = test[['ip', 'app', 'device', 'os', 'channel', 'session']]

In [0]:
# Using the dictionary to label future data
test.apply(lambda x: d[x.name].transform(x))

ValueError: ('y contains new labels: [     0      1      2 ..., 126409 126412 126413]', 'occurred at index ip')

In [0]:
test = test.reset_index(drop = False)
test = test.rename(columns={'index':'click_id'})
test.head()

In [0]:
print(len(test.channel.unique()))
test.channel.value_counts().sort_index()

In [0]:
test.dtypes

In [0]:
test_x_names = test.click_id
test_x = test.drop('click_id', 1)


In [0]:
X_2016 = sc.transform(test_x)

In [0]:
# Predicting on the test set 
predictions = classifier.predict(X_2016)
predictions

In [0]:
pred_clas = classifier.predict_classes(X_2016)

In [0]:
labels_ = predictions.argmax(axis=1)
#labels_ = labels_> 0.5
labels_

In [0]:
a = pd.concat([pd.DataFrame(test_x_names), pd.DataFrame(predictions)], axis =1).rename(columns={0:'is_attributed'})        

In [0]:
a.to_csv('First.csv', header = True, index = False)

In [0]:
a.to_csv('primera.csv', header = True, index = False)

In [0]:
predictions1 = predictions